In [ ]:
import numpy as np
from scipy.interpolate import InterpolatedUnivariateSpline
import matplotlib.pyplot as plt

y = np.sort(np.random.random((1,30)),1)*10

x = np.sort(np.random.random((1_000,30)),1)*10

eval_points = [1.1, 2.2, 3.3, 4.4, 6]

In [ ]:
def single_reconstruction(x,y):
    return InterpolatedUnivariateSpline(x,y).derivative()

# vectorized evaluation to generate the fit spline derivatives
vectorized_reconstructions = np.vectorize(single_reconstruction, cache=True, signature='(locations),(quantiles)->()')

# Hydrate the interpolation functions with x,y values
interpolation_functions = vectorized_reconstructions(x,y)

In [ ]:
interpolation_functions_from_list_comprehension = [InterpolatedUnivariateSpline(x[i,:],y) for i in range(0,len(x[:,0]))]

In [ ]:
for interp_func in interpolation_functions:
    interp_func(eval_points)

In [ ]:
for interp_func in interpolation_functions_from_list_comprehension:
    interp_func(eval_points)

In [ ]:
A = np.asarray([f(eval_points) for f in interpolation_functions])

In [ ]:
A = np.asarray([f(eval_points) for f in interpolation_functions_from_list_comprehension])

In [ ]:
from qp.quantile_pdf_constructors import CdfSplineDerivative

spliner = CdfSplineDerivative(y, x)
spliner.prepare_constructor() # not explicitly necessary, but can get timing information from this. 
output = spliner.construct_pdf(eval_points)

print(len(output))
print(output[0])

# Show the use of the `row` keyword
output = spliner.construct_pdf(eval_points, row=[10, 12])
print(len(output))
print(output)


In [ ]:
# Create a normal distribution. Extract quants and locs, and build a quantile ensemble.
import qp
# Working with a normal distribution
dist2 = qp.stats.norm(loc=3, scale=1)

# Starting with a simple base case. Reasonable quantiles >0, <1.0
new_quants = np.linspace(0.001, 0.999, 16)
new_locs = dist2.ppf(new_quants)

# ! Can't figure out how to pass additional arguments, i.e. `pdf_constructor='cdf_spline_derivative'` ???
quant_dist = qp.quant(quants=new_quants, locs=new_locs)
# Print out the constructor name, and a few p(z) values for a few points
print(quant_dist.dist.pdf_constructor_name)
quant_dist.pdf([1,2,3,4])

In [ ]:
# setting the pdf_constructor_name again produces a warning.
quant_dist.dist.pdf_constructor_name = 'cdf_spline_derivative'

# setting it to something that doesn't exist in the PDF_CONSTRUCTORS raises a ValueError
# commented out to allow all the cells to run. Uncomment this line to see the error.
# quant_dist.dist.interpolator = 'drewtonian'

In [ ]:
qp.plotting.plot_native(quant_dist, xlim=(0, 6))

In [ ]:
# Recalculate the splines using a different spline_order
quant_dist.dist.pdf_constructor_name = 'cdf_spline_derivative'
quant_dist.dist.pdf_constructor.prepare_constructor(spline_order=1)
qp.plotting.plot_native(quant_dist, xlim=(0, 6))

In [ ]:
# change the selected interpolator in quant Ensemble
quant_dist.dist.pdf_constructor_name = 'piecewise_linear'
qp.plotting.plot_native(quant_dist, xlim=(0, 6))

In [ ]:
quant_dist.dist.pdf_constructor_name = 'dual_spline_average'
print(quant_dist.dist.pdf_constructor_name)
qp.plotting.plot_native(quant_dist, xlim=(0,6))

In [ ]:
# showing how to use CdfSplineDerivative outside of a quant Ensemble.
spliner_for_quant_dist = CdfSplineDerivative(new_quants, np.array([new_locs]))
spliner_for_quant_dist.prepare_constructor()
grid = np.linspace(0,6,100)
output = spliner_for_quant_dist.construct_pdf(grid)
plt.plot(grid, np.squeeze(output))